# 객체 검출과 마스킹

## 07-01 얼굴 검출


In [1]:
import cv2

# Haar Cascade 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +
                                     'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출 (scaleFactor=1.1, minNeighbors=5 권장)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('Face Detection (q=quit)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


### 얼굴과 눈 모두 검출

In [2]:
import cv2

# Haar Cascade 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +
                                     'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +
                                    'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # 얼굴 ROI 영역 안에서만 눈 검출
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 3)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)

    cv2.imshow('Face & Eyes Detection (q=quit)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


## 07-02 ROI 모자이크 처리 (얼굴 블러/픽셀화)

In [5]:
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +
                                     'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]

        # --- 모자이크 처리 ---
        small = cv2.resize(roi, (w//15, h//15), interpolation=cv2.INTER_LINEAR)
        mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

        frame[y:y+h, x:x+w] = mosaic  # ROI 교체

    cv2.imshow('Face Mosaic (q=quit)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


### 중급 버전 소스

- 얼굴을 옆으로 기울이거나 측면으로 돌려도 모자이크 유지

In [7]:
import cv2
from collections import deque

# 카스케이드 로드
face_frontal = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
face_profile = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("카메라 불가")

# 최근 박스를 잠깐 기억(깜빡임 방지)
HOLD_FRAMES = 10
last_boxes = deque(maxlen=HOLD_FRAMES)

def detect_faces(gray):
    boxes = []

    # 1) 정면
    faces = face_frontal.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(80,80))
    boxes.extend(faces)

    # 2) 프로파일(왼쪽)
    prof = face_profile.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(80,80))
    boxes.extend(prof)

    # 3) 프로파일(오른쪽) → 좌우반전으로 대응
    flipped = cv2.flip(gray, 1)
    prof_r = face_profile.detectMultiScale(flipped, scaleFactor=1.1, minNeighbors=4, minSize=(80,80))
    w = gray.shape[1]
    # 반전 좌표를 원본 좌표로 되돌림
    for (x, y, fw, fh) in prof_r:
        boxes.append((w - x - fw, y, fw, fh))

    return boxes

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    boxes = detect_faces(gray)

    if boxes:
        # 최신 박스 갱신
        last_boxes.clear()
        last_boxes.extend(boxes)
    else:
        # 최근 박스를 임시 사용(완전 소실 방지)
        boxes = list(last_boxes)

    # 모자이크 처리
    for (x, y, w, h) in boxes:
        # 안전 클리핑
        x = max(0, x); y = max(0, y)
        w = max(1, w); h = max(1, h)
        x2 = min(frame.shape[1], x + w)
        y2 = min(frame.shape[0], y + h)
        roi = frame[y:y2, x:x2]
        if roi.size == 0:
            continue
        small = cv2.resize(roi, (max(1, w//15), max(1, h//15)), interpolation=cv2.INTER_LINEAR)
        mosaic = cv2.resize(small, (x2 - x, y2 - y), interpolation=cv2.INTER_NEAREST)
        frame[y:y2, x:x2] = mosaic

    cv2.imshow('Face Mosaic (q=quit)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## 07-03 HSV 색상 마스크

In [8]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 빨강색 범위 정의 (두 구간 필요)
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 | mask2

    # 마스크 적용
    result = cv2.bitwise_and(frame, frame, mask=mask)

    cv2.imshow("Original", frame)
    cv2.imshow("Red Masked", result)

    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


### 흰색 마스크

In [9]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 흰색 범위 정의
    lower_white = np.array([0, 0, 200])    # H=0~180 전체, S=0~30, V=200~255
    upper_white = np.array([180, 30, 255])

    mask = cv2.inRange(hsv, lower_white, upper_white)

    # 마스크 적용
    result = cv2.bitwise_and(frame, frame, mask=mask)

    cv2.imshow("Original", frame)
    cv2.imshow("White Masked", result)

    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()
